Here we build a GP and sample from it directly on some pre-defined positions.

In [1]:
# importing all the necessary packages
import cPickle as pickle
import numpy as np
import scipy
import GPy
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.linalg 
from scipy.special import cbrt

In [2]:
class PermeabilityModel(object):

    """
    A stochastic model for permeability.

    :param X1:  x coordinates of grid
    :param X2:  y coordinates of grid
    :param X3:  z coordinates of grid
    :param cov: a covariance function
    :param K0:  a geometric mean level for the field
    :param X:   observed inputs
    :param Y:   observed outputs
    """

    def __init__(self, X1, X2, X3, cov, mu0=0., X=None, Y=None, nugget=1e-12):
        self.cov = cov
        self.X1 = X1
        self.X2 = X2
        self.X3 = X3
        n = np.prod(X1.shape)
        self.n = n
        self.mu0 = mu0
        self.X_all = np.hstack([X1.flatten()[:, None], X2.flatten()[:, None], X3.flatten()[:, None]])
        if X is not None:
            CX = cov.K(X) + nugget * np.eye(X.shape[0])
            L = np.linalg.cholesky(CX)
            self.LiY = np.linalg.solve(L, Y - mu0)
            CXaX = cov.K(self.X_all, X)
            self.G0 = self.mu0 + np.dot(CXaX, self.LiY).reshape(self.X1.shape)
            self.C = cov.K(self.X_all) - np.dot(CXaX, CXaX.T) + nugget * np.eye(self.X_all.shape[0])
        else:
            self.C = cov.K(self.X_all) + nugget * np.eye(self.X_all.shape[0])
            self.G0 = np.ones(self.X1.shape) * mu0
        self.L = np.linalg.cholesky(self.C)

    def sample(self):
        """
        Sample the model.
        """
        g = np.dot(self.L, np.random.randn(self.L.shape[0]))
        G = g.reshape(self.X1.shape)
        return self.G0 + G

In [3]:
# problem parameters
x1_L = 250. 
x2_L = 50.
x3_L = 1. # depth
Nx = 15
Ny = 15
Nz = 15   # layers in depth

x1 = np.linspace(0, 1, Nx)
x2 = np.linspace(0, 1, Ny)
x3 = np.linspace(0, 1, Nz)
X1, X2, X3 = np.meshgrid(x1, x2, x3)
k = GPy.kern.Exponential(3, ARD=True, lengthscale=[50./250, 5./50, 1], variance=0.25)
perm_X = PermeabilityModel(X1, X2, X3, k, mu0=-23)

In [4]:
G = perm_X.sample() # Sampled log permeability
print G.shape
print np.mean(G.flatten())
print np.mean(np.log10(np.exp(G.flatten())))
print np.var(G.flatten())

(15, 15, 15)
-22.986017001884434
-9.982700344852736
0.20822739201674728


In [10]:
# Sampling is fast
Kxs = []
for i in xrange(5):
    Kx = np.exp(perm_X.sample())
#     print i
    Kxs.append(Kx)
# Kxs[0].shape

In [9]:
# Visualize at a specific x y location as a function of depth
# plt.plot(Kxs[0][1, :, 0])

In [8]:
# Visualize one sample for various depths
def plot_contour(depth=0):
    fig, ax = plt.subplots()
#     c = ax.contourf(X1[:, :, 0], X2[:, :, 0], Kxs[0][:, :, depth])
    c = ax.contourf(X1[:, :, 0], X2[:, :, 0], np.log10(Kxs[4][:, :, depth]))

    plt.colorbar(c)
    ax.set_aspect(0.2)    
from ipywidgets import interactive
interactive(plot_contour, depth=(0, 14, 1))

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2RlcHRoJywgbWF4PTE0KSwgT3V0cHV0KCkpLCBfZG9tX2NsYXNzZXM9KHUnd2lkZ2V0LWludGXigKY=


In [47]:
well_num = 2
dim = 3
# we load well data here
well1_data = np.loadtxt('data/well1.txt') #depth Kx Ky Kz porosity 
well2_data = np.loadtxt('data/well2.txt') #depth Kx Ky Kz porosity 
#converting well data 
well1_data[:,1:-1] = np.log10(1e-3*9.869233e-13*well1_data[:,1:-1])
well1_data[:,4] = (1e-2*well1_data[:,4])
well2_data[:,1:-1] = np.log10(1e-3*9.869233e-13*well2_data[:,1:-1])
well2_data[:,4] = (1e-2*well2_data[:,4])
# we model the fixed depth reservoir, thus we eliminate the extra information for well2 (min function)
num_obs = well_num*min(len(well1_data),len(well2_data)) #if more wells add here
X_obs = np.zeros ((num_obs, dim)) # (num_obs, dim) r = x1,x2,x3   
Y_obs = np.zeros ((num_obs, 4))  # (num_obs, unknown parameters:Kx,Ky,Kz,porosity)                 
# calculating cell-centers
xc = np.zeros((Nx,Ny,Nz))
yc = np.zeros((Nx,Ny,Nz))
zc = np.zeros((Nx,Ny,Nz))
for i in range(0,Nx):
  for j in range(0,Ny):
    for k in range(0,Nz):
        xc[i,j,k] = (i+0.5)*x1_L/Nx/x1_L #we divide by x1_L to normalize xc
        yc[i,j,k] = (j+0.5)*x2_L/Ny/x2_L
        zc[i,j,k] = (k+0.5)*x3_L/Nz/x3_L
# we construct the observation coordinates and values here in X_obs and Y_obs, respectively 
# well1[i,j,k] --> 0,0,0-14, well2[i,j,k] --> 9,9,0-14
for k in range(0,int(0.5*num_obs)): # we sweep along well1 in z
    X_obs[k,:] = [xc[0,0,k],yc[0,0,k],zc[0,0,k]]
    Y_obs[k,:] = well1_data[k,1:5]
for k in range(int(0.5*num_obs),num_obs): # we sweep along well2 in z
    X_obs[k,:] = [xc[9,9,k-int(0.5*num_obs)],yc[9,9,k-int(0.5*num_obs)],zc[9,9,k-int(0.5*num_obs)]]
    Y_obs[k,:] = well2_data[k-int(0.5*num_obs),1:5]
print X_obs
print Y_obs

[[ 0.05        0.05        0.03333333]
 [ 0.05        0.05        0.1       ]
 [ 0.05        0.05        0.16666667]
 [ 0.05        0.05        0.23333333]
 [ 0.05        0.05        0.3       ]
 [ 0.05        0.05        0.36666667]
 [ 0.05        0.05        0.43333333]
 [ 0.05        0.05        0.5       ]
 [ 0.05        0.05        0.56666667]
 [ 0.05        0.05        0.63333333]
 [ 0.05        0.05        0.7       ]
 [ 0.05        0.05        0.76666667]
 [ 0.05        0.05        0.83333333]
 [ 0.05        0.05        0.9       ]
 [ 0.05        0.05        0.96666667]
 [ 0.95        0.95        0.03333333]
 [ 0.95        0.95        0.1       ]
 [ 0.95        0.95        0.16666667]
 [ 0.95        0.95        0.23333333]
 [ 0.95        0.95        0.3       ]
 [ 0.95        0.95        0.36666667]
 [ 0.95        0.95        0.43333333]
 [ 0.95        0.95        0.5       ]
 [ 0.95        0.95        0.56666667]
 [ 0.95        0.95        0.63333333]
 [ 0.95        0.95      

In [ ]:
class KarhunenLoeveExpansion(object):
    
    """
    A class representing the Karhunen Loeve Expansion of a Gaussian random field.
    It uses the Nystrom approximation to do it.
    
    Arguments:
        k      -     The covariance function.
        Xq     -     Quadrature points for the Nystrom approximation.
        wq     -     Quadrature weights for the Nystrom approximation.
        alpha  -     The percentage of the energy of the field that you want to keep.
        X      -     Observed inputs (optional).
        Y      -     Observed field values (optional).
    """
    
    def __init__(self, k, Xq=None, wq=None, nq=100, alpha=0.9, X=None, Y=None):
        self.k = k
        if Xq is None:
            if k.input_dim == 1:
                Xq = np.linspace(0, 1, nq)[:, None]
                wq = np.ones((nq, )) / nq
            elif k.input_dim == 2:
                #nq = int(np.sqrt(nq))
                x = np.linspace(0, 1, nq)
                X1, X2 = np.meshgrid(x, x)
                Xq = np.hstack([X1.flatten()[:, None], X2.flatten()[:, None]])
                wq = np.ones((nq ** 2, )) / nq ** 2
            elif k.input_dim == 3:
                #nq = int(cbrt(nq))
                x = np.linspace(0, 1, nq)
                X1, X2, X3 = np.meshgrid(x, x, x)
                Xq = np.hstack([X1.flatten()[:, None], X2.flatten()[:, None], X3.flatten()[:, None]])
                wq = np.ones((nq ** 3, )) / nq ** 3                
            else:
                raise NotImplementedError('For more than 3D, please supply quadrature points and weights.')
        self.Xq = Xq
        self.wq = wq
        self.k = k
        self.alpha = alpha
        self.X = X
        self.Y = Y
        # If we have some observed data, we need to use the posterior covariance
        if X is not None:
            self.K = self.k.K(X) + 1e-12 * np.eye(X.shape[0])
            self.L = np.linalg.cholesky(self.K)
            self.LiY = np.linalg.solve(self.L, Y)
            Kqp = self.k.K(Xq)
            Kcp = self.k.K(X, Xq)
            self.LiKcp = np.linalg.solve(self.L, Kcp)
            Kq = Kqp - np.dot(self.LiKcp.T, self.LiKcp)
            #gpr = GPy.models.GPRegression(X, Y[:, None], k)
            #gpr.likelihood.variance = 1e-12
            #self.gpr = gpr
            #Kq = gpr.predict(Xq, full_cov=True)[1]
        else:
            Kq = k.K(Xq)
        B = np.einsum('ij,j->ij', Kq, wq)
        lam, v = scipy.linalg.eigh(B, overwrite_a=True)
        lam = lam[::-1]
        lam[lam <= 0.] = 0.
        energy = np.cumsum(lam) / np.sum(lam)
        i_end = np.arange(energy.shape[0])[energy > alpha][0] + 1
        lam = lam[:i_end]
        v = v[:, ::-1]
        v = v[:, :i_end]
        self.lam = lam
        self.sqrt_lam = np.sqrt(lam)
        self.v = v
        self.energy = energy
        self.num_xi = i_end
        self.x_prev = None

    def eval_phi(self, x):
        """
        Evaluate the eigenfunctions at x.
        """
        if self.X is not None:
            KXx = self.k.K(self.X, x)
            LiKXx = np.linalg.solve(self.L, KXx)
            m = np.dot(LiKXx.T, self.LiY)
            Kc = self.k.K(x, self.Xq) - np.dot(LiKXx.T, self.LiKcp)
            self.tmp_mu = m
        else:
            Kc = self.k.K(x, self.Xq)
            self.tmp_mu = 0.
        phi = np.einsum('i,ji,j,rj->ri', 1. / self.lam, self.v, self.wq, Kc)
        return phi
    
    def __call__(self, x, xi):
        """
        Evaluate the expansion at x and xi.
        """
        if self.x_prev is not x:
            self.x_prev = x
            self.tmp_phi = self.eval_phi(x)
        print self.tmp_mu
        return self.tmp_mu + np.dot(self.tmp_phi, xi * self.sqrt_lam)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# First, let's fit a Gaussian process to the data.
# This will automatically determine the best lengthscale and variance by maximizing the likelihood
k1 = GPy.kern.Exponential(3, ARD=True)
k1.lengthscale.constrain_fixed([0.1, 0.1, 0.1])
k_permX = k1
# There are not enough data to find the X and Y length scales. We need to fix their values to the 
# persumed variability.
# Center the data about 0
m0 = Y_obs.mean(axis=0)
model_permX = GPy.models.GPRegression(X_obs, Y_obs[:, 0:1] - m0[0], k_permX)
# We do not want the field to have any noise
model_permX.likelihood.variance.constrain_fixed(1e-12)
# Let the GP optimize for just the variance of the kernel
model_permX.optimize()
print str(model_permX)

In [ ]:
# Now from this, you can actually take samples at any grid you want.
# For relatively small grids. This is because the following actually sets up and does Cholesky on a
# (nx x ny x nz) x (nx x ny x nz) matrix... 
#Kx = model_permX.posterior_samples(X_all, 10)
# Each column is a sample
#print Kx.shape

In [ ]:
# Alternatively, let's use the Nystrom to the KLE.
# nq controls the accuracy of the approximation (number of quadrature points per dimension)
# We must start low and keep increasing it, until the approximation does not change
fig, ax = plt.subplots()
for nq in [2, 4, 8, 16, 32]:
    # Notice that here I give the kernel with the optimized parameters
    kle_permX = KarhunenLoeveExpansion(model_permX.kern, nq=nq, alpha=0.95, X=X_obs, Y=Y_obs[:,0] - m0[0])
    print 'nq = {0:d}, num_terms = {1:d}'.format(nq, kle_permX.num_xi)
    ax.plot(kle_permX.lam)

In [ ]:
Kx = kle_permX(X_all, np.random.randn(kle_permX.num_xi)) + m0[0] # Remember to add back the constant mean
print Kx

In [ ]:
# The second time must bef much faster - Maybe save the object in the pickle file after the first evaluation.
Kx = kle_permX(X_all, np.random.randn(kle_permX.num_xi))
print Kx

In [ ]:
plt.contourf(X1[:, :, 0], X2[:, :, 0], Kx.reshape(X1.shape)[:, :, 0])

In [ ]:
# The second time must bef much faster:
Kxs = []
for i in xrange(5):
    #Kx = kle_permX(X_all, np.random.randn(kle_permX.num_xi)) + m0[0]
    Kx = model_permX.posterior_samples_f(X_all)[:,0] + m0[0]
    print i
    Kxs.append(Kx)

In [ ]:
def plot_contour(i=0):
    for Kx in Kxs[:1]:
        fig, ax = plt.subplots()
        c = ax.contourf(X1[:, :, 0], X2[:, :, 0], Kx.reshape(X1.shape)[:, :, i])
        plt.colorbar(c)
        
from ipywidgets import interactive
interactive(plot_contour, i=(0, 14, 1))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots()
# the parameter nq controls the accuracy of the Nystrom approximation.
k_permX = GPy.kern.Exponential(3, ARD=True)
# First, let's 

# You should start low and gradually increase it until it does not change a lot.
kle_permX = KarhunenLoeveExpansion(k_permX, nq=60, alpha=0.95, X=X_obs, Y=Y_obs[:,0])
ax.plot(kle_permX.lam)
#handler_permX = open('permX', 'wb')
#pickle.dump(kle_permX, handler_permX) 
with open('permX8.pcl', 'wb') as fp:
    pickle.dump(kle_permX, fp)
#handler_permX.close()
#kle_permY = KarhunenLoeveExpansion(k, nq=512, alpha=0.95, X=X_obs, Y=Y_obs[:,1])
#handler_permY = open('permY', 'wb')
#pickle.dump(kle_permY, handler_permY) 
#kle_permZ = KarhunenLoeveExpansion(k, nq=512, alpha=0.95, X=X_obs, Y=Y_obs[:,2])
#handler_permZ = open('permZ', 'wb')
#pickle.dump(kle_permZ, handler_permZ)
#k_poro = GPy.kern.Exponential(3, ARD=True)
#k_poro = GPy.kern.RBF(3, ARD=True)
#kle_poro  = KarhunenLoeveExpansion(k_poro, nq=512, alpha=0.95, X=X_obs, Y=Y_obs[:,3])
#handler_poro  = open('poro' , 'wb')
#pickle.dump(kle_poro, handler_poro)

print 'Number of terms for permX:', kle_permX.num_xi
#print 'Number of terms for permY:', kle_permY.num_xi
#print 'Number of terms for permZ:', kle_permZ.num_xi
#print 'Number of terms for poro :', kle_poro.num_xi